In [26]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Latex, Markdown, HTML
import sys
import locale

sys.path.append('.')
import vaccines as lib

# Dados e configurações
plt.rcParams["figure.figsize"] = (10,6.66)
pd.set_option('display.max_rows', None)

for loc in ['pt_BR.UTF-8', '']:
    try:
        locale.setlocale(locale.LC_ALL, loc)
        break
    except:
        pass

data = pd.read_pickle('data.pkl.gz')
hospitalization = pd.read_pickle('hospitalization.pkl.gz')
deaths = pd.read_pickle('deaths.pkl.gz')

In [33]:
voila = False
if voila:
    display(HTML("""
<style>
    body {
        max-width: 45rem !important;
        margin: auto !important;
    }
    .output_area:first-child {
        position: fixed !important;
        top: 6rem !important;
        right: 4rem !important;
        min-width: 10rem !important; 
    }
</style>
"""))
display(Markdown("## Ferramenta para determinação do impacto da vacinação nas internações por COVID-19"))
default_plan = """95%"""
region_options = [(v, k) for k, v in data['name'].to_dict().items()]
vaccine_options = [('Coronavac/sinovac/butantan', (22, 28, 0.95)), ('Astrazeneca/Fiocruz', (90, 28, 0.95))]


@widgets.interact(
    region=widgets.Dropdown(options=region_options, description="UF"),
    stock=widgets.IntText(500_000, description="Doses"),
    rate=widgets.IntText(6_000, description="Capacidade"),
    vaccine=widgets.Dropdown(options=vaccine_options, description="Tipo vacina"),
    plan=widgets.Textarea(default_plan, description="Metas"),
    coarse=widgets.Checkbox(False, description='Agrupar de 10 em 10 anos'),
)
def run(stock, region, rate, vaccine, plan, coarse):
    global events, age_distribution, severe, result
    
    if stock == 0:
        display(Markdown('**ERRO:** Selecione a quantidade de vacinas!'))
        return 

    second_dose_delay, immunization_delay, eff = vaccine
    rates = lib.compute_schedule(rate, stock)
    age_distribution = lib.population_80_plus(data.loc[region, "age_distribution"], coarse=coarse)
    
    plan = lib.SimpleDosesRatePlan.from_source(plan, age_distribution, rate=rate, delay=second_dose_delay, max_doses=stock // 2)
    plan.execute()
    events = plan.summary()
    
    severe = (hospitalization.loc[region].iloc[::-1])
    death = (deaths.loc[region].iloc[::-1])
    if coarse:
        severe = lib.coarse_distribution(severe)
        death = lib.coarse_distribution(death)
        
    duration = lib.by_periods(events['day'].max() + immunization_delay + second_dose_delay, 30)
    result = lib.VaccinationCampaign(events, duration=duration)
    
    h_pressure = result.damage_curve(severe, delay=immunization_delay, efficiency=eff)
    d_pressure = result.damage_curve(death, delay=immunization_delay)
    display(Markdown(f"""
## Resultados
* **Total de doses:** {int(result.vaccines):n}
* **Pessoas vacinadas:** {int(result.vaccines // 2):n}
* **Dias de vacinação:** {result.campaign_duration}
* **Redução na hospitalização:** {100 - 100 * h_pressure.iloc[-1]:.1f}%
* **Redução dos óbitos:** {100 - 100 * d_pressure.iloc[-1]:.1f}%
    """))
    
    result.plot_hospitalization_pressure_curve(h_pressure, as_pressure=True)
    plt.show()
    
    result.plot_death_pressure_curve(d_pressure, as_pressure=True)
    plt.show()
    
    result.plot_vaccination_schedule()
    plt.show()
    
       
    display(Markdown(f"""
## Observações

O primeiro gráfico mostra a estimativa de redução nas hospitalizações esperadas enquanto o segundo gráfico mostra a estimativa de redução de mortalidade, em função da estratégia de imunização, ou seja devido à proteção conferida pelas vacinas. 

O terceiro gráfico mostra a cobertura vacinal por faixa etária ao longo do tempo. A faixa cinza mais clara representa o percentual da população que ainda não foi imunizada, 
a faixa cinza mais escura mostra a população que está aguardando a segunda dose e a faixa verde mostra a população que na qual foram aplicadas as duas
doses da vacina. 

A simulação considera que a taxa de infecção se mantêm constante, o que é uma suposição conservadora, 
especialmente em níveis mais altos de vacinação.

## Parâmetros utilizados 

1. Eficácia para formas graves da doença COVID-19 = 100%
2. Para o esquema vacinal foram considerados os seguintes intervalos:
    - **Astrazeneca/Fiocruz:** O esquema de imunização é de 2 doses com intervalo máximo de 90 dias entre as doses.
    - **Butantan:** O esquema de imunização é de 2 doses com intervalo máximo de 22 dias entre as doses.
3. Foram considerados imunizados apenas os indivíduos que tiverem as duas doses da vacina, 
4. A imunização (soroconversão) ocorre em um período de 28 dias após aplicação da segunda dose

## Referências

Parecer Público de avaliação de solicitação de autorização temporária de uso
emergencial, em caráter experimental, da vacina adsorvida covid-19 (inativada) –
[Instituto Butantan](https://www.gov.br/anvisa/pt-br/assuntos/noticias-anvisa/2021/confira-materiais-da-reuniao-extraordinaria-da-dicol/ppam-final-vacina-adsorvida-covid-19-inativada-butantan.pdf)

Parecer Público de avaliação de solicitação de autorização temporária de uso
emergencial, em caráter experimental, da vacina covid-19 (recombinante) –
[Fundação Oswaldo Cruz (Fiocruz)](https://www.gov.br/anvisa/pt-br/assuntos/noticias-anvisa/2021/confira-materiais-da-reuniao-extraordinaria-da-dicol/ppam-final-vacina-covid-19-recombinante-fiocruz.pdf)
    """))


## Ferramenta para determinação do impacto da vacinação nas internações por COVID-19

interactive(children=(IntText(value=500000, description='Doses'), Dropdown(description='UF', options=(('Acre',…

In [9]:
for m in list(sys.modules):
    if m.startswith('vaccines'):
        del sys.modules[m]
import vaccines as lib

from typing import List, Dict, Tuple, NamedTuple, Iterator
from vaccines import Plan, Event
from collections import defaultdict, Counter

# plan = lib.MultipleVaccinesRatePlan.from_source('90%', age_distribution, rates=[2000, 1000], delays=[22, 90], max_doses=[20000, 10000])
# plan.execute()
# plan.summary().iloc[:25]

In [32]:
src = '''
# Informe as doses já aplicadas por faixa etária
80: 0
75: 0
70: 0
65: 0
...
'''
lib.parse_plan(src, age_distribution)

[(80, 100), (75, 2000), (70, 2000)]